# Microsoft PromptFlow

Comme [LangChain](./langchain.ipynb) et [LlamaIndex](./llamaindex.ipynb), Microsoft PromptFlow founit un framework pour la création de pipelines RAG, mais en plus PromptFlow fournit une architecture de déploiement et des outils de trace et d'évaluation de type LangSmith.

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/promptflow.ipynb)

Docs:
- https://github.com/microsoft/promptflow
- https://microsoft.github.io/promptflow/

In [36]:
import os
import sys

# Supprimer les commentaires pour installer (requirements.txt)
# !pip3 install -q -U python-dotenv
# !pip3 install -q -U "promptflow[executable]"

# À installer dans tous les cas pour Google Colab et Github
if 'google.colab' in sys.modules or 'CI' in os.environ:
    !pip3 install -q -U openai
    !pip3 install -q -U promptflow
    !pip3 install -q -U promptflow-tools

Considérer éventuellement l'installation de l'extension pour VS Code:
- https://marketplace.visualstudio.com/items?itemName=prompt-flow.prompt-flow

In [37]:
# Les variables python sont accessibles depuis les commandes shell
flow_name = 'promptflow'
work_directory = f'./{flow_name}'

#!mkdir -p $work_directory # Inutile, le répertoire est créé par `pf flow init` ci-après

## Création d'un flux standard basique

Un flux standard basique comprend essentiellement un modèle de prompt au format `.jinja2`, un fichier de code python `.py`, et un fichier `flow.dag.yaml`décrivant le DAG (Directed Acyclic Graph) du flux, qui dans ce cas consiste à fusionner le texte d'entrée avec le modèle pour produire un prompt qui est passé à la fonction du fichier de code python.

In [38]:
# Création du répertoire et des fichiers du flux
!pf flow init --flow $flow_name --yes

/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Creating flow from scratch...
Creating hello.py...
Creating data.jsonl...
Creating .promptflow folder...
Creating flow.dag.yaml...
Creating hello.jinja2...
Creating /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/tools/promptflow/requirements.txt...
Creating /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/tools/promptflow/.gitignore...
Done. Created standard flow folder: /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/tools/promptflow.
You can execute this command to test the flow, pf flow test --flow promptflow --input promptflow/data.jsonl


In [39]:
# Exécution du flux qui retourne le prompt résultant de la fusion du modèle jinja2 et du texte dans `data.jsonl`.
!pf flow test --flow $flow_name --input $work_directory/data.jsonl

/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Prompt flow service has started...
You can view the traces from local: http://localhost:51053/v1.0/ui/traces/?#collection=promptflow
2024-04-24 20:54:55 +0200    4248 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-04-24 20:54:55 +0200    4248 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-04-24 20:54:55 +0200    4248 execution.flow     INFO     Executing node hello_prompt. node run id: b414c817-4080-419a-87dc-c40833aae9f7_hello_prompt_0
2024-04-24 20:54:55 +0200    4248 execution.flow     INFO     Node hello_prompt completes.
2024-04-24 20:54:55 +0200    4248 execution.flow     INFO     Executing node echo_my_prompt. node run id: b414c817-4080-419a-87dc

## Intégration d'un LLM (OpenAI)

### Changement de modèle de prompt

Créons un prompt pour faire de l'analyse de sentiment.

In [40]:
%%writefile $work_directory/hello.jinja2
Fais l'analyse de sentiment du texte délimité par un triple guillemet simple.
Réponds en un seul mot: soit "positif", soit "négatif", soit "neutre".
Réponds "neutre" si tu ne sais pas. Réponds "négatif" si le langage est toxique.

Texte à évaluer: '''{{text}}'''

Overwriting ./promptflow/hello.jinja2


### Changement de la fonction de traitement du prompt

Créons une fonction qui passe le prompt à l'API d'OpenAI.

In [41]:
%%writefile $work_directory/hello.py
import os
import sys
from promptflow.core import tool
from openai import OpenAI

@tool
def my_python_tool(input1: str) -> str:
    openai_api_key = None
    if 'google.colab' in sys.modules:
        from google.colab import userdata
        openai_api_key = userdata.get('OPENAI_API_KEY')
    else:
        from dotenv import load_dotenv, find_dotenv
        _ = load_dotenv(find_dotenv()) # lire le fichier .env local
        openai_api_key  = os.environ['OPENAI_API_KEY']
    client = OpenAI(api_key=openai_api_key)
    completion = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[{'role': 'user', 'content': input1}]
    )
    return completion.choices[0].message.content

Overwriting ./promptflow/hello.py


### Changement du fichier de données entrantes

In [42]:
%%writefile $work_directory/data.jsonl
{"text": "Quelle belle voiture!"}

Overwriting ./promptflow/data.jsonl


In [43]:
# Exécution du flux qui retourne le sentiment ressortisant du texte dans `data.jsonl`.
!pf flow test --flow $flow_name --input $work_directory/data.jsonl

/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Prompt flow service has started...
You can view the traces from local: http://localhost:51053/v1.0/ui/traces/?#collection=promptflow
2024-04-24 20:54:57 +0200    4256 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-04-24 20:54:57 +0200    4256 execution.flow     INFO     Start to run 2 nodes with concurrency level 16.
2024-04-24 20:54:57 +0200    4256 execution.flow     INFO     Executing node hello_prompt. node run id: 4847e2b3-35a8-457a-8889-561449517a28_hello_prompt_0
2024-04-24 20:54:57 +0200    4256 execution.flow     INFO     Node hello_prompt completes.
2024-04-24 20:54:57 +0200    4256 execution.flow     INFO     Executing node echo_my_prompt. node run id: 4847e2b3-35a8-457a-8889

In [44]:
# N'exécuter que dans VS Code (au risque de bloquer les tests CI)
if (len(sys.argv) == 2):
    !pf flow serve --source $work_directory --port 8080 --host localhost

/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
[2024-04-24 20:55:00,481][pfserving-app][INFO] - opentelemetry-instrumentation-flask is not installed, auto-instrumentation is not enabled.
[2024-04-24 20:55:00,481][pfserving-app][INFO] - Static_folder: /Users/jlchereau/Library/Python/3.9/lib/python/site-packages/promptflow/core/_serving/static
[2024-04-24 20:55:00,481][pfserving-app][INFO] - Project path: /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/tools/promptflow
[2024-04-24 20:55:00,485][pfserving-app][INFO] - No connection string detected, app insight metric exporter is disabled.
[2024-04-24 20:55:00,485][pfserving-app][INFO] - No connection string detected, app insight trace exporter is disabled.
[2024-04-24 20:55:00,485][pfserving-app][WARNING

Testez le serveur avec la commande suivante dans une fenêtre de terminal.

```
curl http://localhost:8080/score --data '{"text":"Quelle belle voiture!"}' -X POST  -H "Content-Type: application/json"
```

Pour "chatter" avec le serveur, naviguez à l'adresse http://localhost:8080.

> <span style='color:red'>Pensez à stopper manuellement l'exécution de la cellule contenant la commande `pf flow serve`.</span>

## Déploiement en production

In [45]:
# Création d'un environnement docker pour `docker build`
!pf flow build --source $work_directory --output $work_directory/build --format docker
# !docker build -t $image_name $work_directory/build

/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jlchereau/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Exported flow to /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/tools/promptflow/build.
please check /Users/jlchereau/WebstormProjects/Digillia/Digillia-Colab/to

In [46]:
# Ménage
!rm -rf $work_directory